In [31]:
import numpy as np
import matplotlib.pyplot as plt
with np.load("C:\Shared_archcraft\SML\Assignment4\mnist.npz") as data:
    x_train, y_train = data['x_train'], data['y_train']
    x_test, y_test = data['x_test'], data['y_test']

train_mask = np.isin(y_train, [0])
x_train_0 = x_train[train_mask]
y_train_0 = [0]*len(x_train_0)

train_mask = np.isin(y_train, [1])
x_train_1 = x_train[train_mask]
y_train_1 = y_train[train_mask]

train_mask = np.isin(y_train, [0, 1])
x_train_01 = x_train[train_mask]
y_train_01 = y_train[train_mask]

print(len(y_train_0))
print(len(y_train_1))


x_test_1 = x_train_1[:1000]
y_test_1 = y_train_1[:1000]
x_test_0 = x_train_0[:1000]
y_test_0 = y_train_0[:1000]

x_train_0 = x_train_0[1000:]
y_train_0 = y_train_0[1000:]
x_train_1 = x_train_1[1000:]
y_train_1 = y_train_1[1000:]

x_real_train = np.concatenate([x_train_0, x_train_1], axis=0)
y_real_train = np.concatenate([y_train_0, y_train_1], axis=0)
x_real_test = np.concatenate([x_test_0, x_test_1], axis=0)
y_real_test = np.concatenate([y_test_0, y_test_1], axis=0)

# Reshape x_real_train
# x_real_train = x_real_train.reshape(-1, 28, 28)
print(y_real_train.shape)
print(x_real_train.shape)

def pca(x):
    X_mean = np.mean(x, axis=0)
    X_centered = x - X_mean
    X_centered_2d = X_centered.reshape(X_centered.shape[0], -1)  # Ensure X_centered is 2D
    cov_matrix = np.cov(X_centered_2d, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    p = 5
    top_indices = np.argsort(eigenvalues)[::-1][:p]
    pca_matrix = eigenvectors[:, top_indices]
    
    x_reduced = np.dot(X_centered_2d, pca_matrix)
    
    print(x_reduced.shape)
    return x_reduced

# pca(x_real_train)
NumOfStumps = 30

5923
6742
(10665,)
(10665, 28, 28)


In [32]:
class DecisionTree():
    def __init__(self, x, y, idxs = None, min_leaf=2):
        if idxs is None: idxs=np.arange(len(y))
        self.x,self.y,self.idxs,self.min_leaf = x,y,idxs,min_leaf
        self.n,self.c = len(idxs), x.shape[1]
        self.val = np.mean(y[idxs])
        self.score = float('inf')
        self.find_varsplit()
        
    def find_varsplit(self):
        for i in range(self.c): self.find_better_split(i)
        if self.score == float('inf'): return
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        self.lhs = DecisionTree(self.x, self.y, self.idxs[lhs])
        self.rhs = DecisionTree(self.x, self.y, self.idxs[rhs])

    def find_better_split(self, var_idx):
        x, y = self.x[self.idxs, var_idx], self.y[self.idxs]
        sort_idx = np.argsort(x)
        sort_y, sort_x = y[sort_idx], x[sort_idx]
        rhs_cnt, rhs_sum, rhs_sum2 = self.n, sort_y.sum(), (sort_y ** 2).sum()
        lhs_cnt, lhs_sum, lhs_sum2 = 0, 0., 0.

        for i in range(0, self.n - self.min_leaf - 1):
            xi, yi = sort_x[i], sort_y[i]
            lhs_cnt += 1
            rhs_cnt -= 1
            lhs_sum += yi
            rhs_sum -= yi
            lhs_sum2 += yi ** 2
            rhs_sum2 -= yi ** 2
            if i < self.min_leaf or xi == sort_x[i + 1]:
                continue

            # lhs_std = std_agg(lhs_cnt, lhs_sum, lhs_sum2)
            # rhs_std = std_agg(rhs_cnt, rhs_sum, rhs_sum2)
            curr_score = lhs_cnt + rhs_cnt
            if curr_score < self.score:
                self.var_idx, self.score, self.split = var_idx, curr_score, xi


    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self):
        return self.x[self.idxs, self.var_idx]


    @property
    def is_leaf(self): return self.score == float('inf')
    
    def __repr__(self):
        s = f'n: {self.n}; val:{self.val}'
        if not self.is_leaf:
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s

    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)
   


In [33]:
x = pca(x_real_test)# initialization of input
y = y_real_test # initialization of target
# x,y --> use where no need to change original y
predf = 0 # initial prediction 0
xi = x # initialization of input
yi = y # initialization of target
# x,y --> use where no need to change original y
ei = 0 # initialization of error
n = len(yi)  # number of rows
predf = 0 # initial prediction 0

for i in range(30): # like n_estimators
    tree = DecisionTree(xi,yi)
    tree.find_better_split(0)
    
    r = np.where(xi == tree.split)[0][0]    
    
    left_idx = np.where(xi <= tree.split)[0]
    right_idx = np.where(xi > tree.split)[0]
    
    predi = np.zeros(n)
    np.put(predi, left_idx, np.repeat(np.mean(yi[left_idx]), r))  # replace left side mean y
    np.put(predi, right_idx, np.repeat(np.mean(yi[right_idx]), n-r))  # right side mean y
    
    predi = predi[:,None]  # make long vector (nx1) in compatible with y
    predf = predf + predi  # final prediction will be previous prediction value + new prediction of residual
    
    ei = y - predf  # needed originl y here as residual always from original y    
    yi = ei # update yi as residual to reloop
    mse = np.mean(ei**2)
    print(f"Iteration {i+1}, MSE: {mse}")

 
    

(2000, 5)
Iteration 1, MSE: 0.2500000225135075


KeyboardInterrupt: 